# Image Recognition Project - Structural Defect Recognition
---------------------------------------------------------------
## Data Visualisation

### Section Objectives
 - Complete the project objective and answer the business requirements by training the -

 --------------------------------------------------------------
 ### Inputs
 inputs/cracks_dataset_new/test
  inputs/cracks_dataset_new/train
   inputs/cracks_dataset_new/validation

----------------------------------------------------------------
### Outputs


## Import Libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from matplotlib.image import imread
from tensorflow.keras.preprocessing import image
import joblib

sns.set_style("whitegrid")

## Setting Working Directory

In [ ]:
cwd= os.getcwd()

In [ ]:
os.chdir('/workspaces/ML_Project_Image_Recognition')
print("You set a new current directory")

In [ ]:
work_dir = os.getcwd()
work_dir

## Setting Input and Output Directories

### Input:

In [ ]:
base_path = 'inputs/cracks_dataset_new' 

train_path = os.path.join(base_path, 'train')
val_path = os.path.join(base_path, 'val')
test_path = os.path.join(base_path, 'test')

datasets = os.listdir(train_path)
print("Detected datasets in training set:", datasets)


### Output:

In [ ]:
version = 'v1'
output_path = os.path.join('outputs', version)

if not os.path.exists(output_path):
    os.makedirs(output_path)
    print(f"Created output folder at: {output_path}")
else:
    print(f"Output folder already exists: {output_path}")

Setting Dataset Labels

In [ ]:
dataset_labels = {}

for dataset in datasets:
    dataset_path = os.path.join(train_path, dataset)
    if os.path.isdir(dataset_path):
        labels = os.listdir(dataset_path)
        dataset_labels[dataset] = labels

print("Dataset → Classes:")
for k, v in dataset_labels.items():
    print(f"• {k}: {v}")

-

In [ ]:
def plot_label_distribution(base_path, dataset_name, save_path=None):
    label_counts = {}

    dataset_path = os.path.join(base_path, dataset_name)
    for label in os.listdir(dataset_path):
        label_path = os.path.join(dataset_path, label)
        if os.path.isdir(label_path):
            label_counts[label] = len([
                f for f in os.listdir(label_path)
                if f.lower().endswith(('.jpg', '.jpeg', '.png'))
            ])

    # Plot
    fig, ax = plt.subplots()
    sns.barplot(x=list(label_counts.keys()), y=list(label_counts.values()), ax=ax)
    ax.set_title(f"Label Distribution in {dataset_name}")
    ax.set_xlabel("Class")
    ax.set_ylabel("Number of Images")

    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"Saved label distribution plot for {dataset_name} at {save_path}")
    else:
        plt.show()

In [ ]:
for dataset in ['Walls', 'Decks', 'Pavements']:
    plot_label_distribution(
        base_path=os.path.join(train_path),
        dataset_name=dataset,
        save_path=os.path.join(output_path, f"label_distribution_{dataset}.png")
    )

-

In [ ]:
def plot_image_dimensions(dataset_name):
    dim1, dim2 = [], []
    print(f"\n Processing image sizes for: {dataset_name}")

    for label in dataset_labels[dataset_name]:  
        class_path = os.path.join(train_path, dataset_name, label)
        for img_file in os.listdir(class_path):
            img_path = os.path.join(class_path, img_file)
            try:
                img = imread(img_path)
                h, w = img.shape[:2]
                dim1.append(h)
                dim2.append(w)
            except:
                continue  

    fig, ax = plt.subplots()
    sns.scatterplot(x=dim2, y=dim1, alpha=0.2)
    ax.set_xlabel("Width (px)")
    ax.set_ylabel("Height (px)")
    ax.set_title(f"Image Size Distribution — {dataset_name}")

    mean_w = int(np.mean(dim2))
    mean_h = int(np.mean(dim1))
    ax.axvline(mean_w, color='r', linestyle='--', label=f"Mean Width: {mean_w}px")
    ax.axhline(mean_h, color='g', linestyle='--', label=f"Mean Height: {mean_h}px")
    plt.legend()
    plt.show()

    return mean_h, mean_w

In [ ]:
walls_shape = plot_image_dimensions('Walls')
decks_shape = plot_image_dimensions('Decks')
pavements_shape = plot_image_dimensions('Pavements')

In [ ]:
image_shape = (256, 256, 3)
print("Image shape to be saved:", image_shape)

joblib.dump(image_shape, os.path.join(output_path, "image_shape.pkl"))
print(f"Image shape saved to: {output_path}/image_shape.pkl")

Function to load images into Array

In [ ]:
from tensorflow.keras.preprocessing import image

def load_image_as_array(data_dir, dataset_name, image_shape=(256, 256), n_images_per_label=30, cache=True):
    cache_file = os.path.join(output_path, f"{dataset_name}_Xy_cache_{n_images_per_label}.pkl")

    if cache and os.path.exists(cache_file):
        print(f"Loading cached data for {dataset_name} from {cache_file}")
        return joblib.load(cache_file)

    X, y = [], []
    labels = dataset_labels[dataset_name]

    for label in labels:
        label_path = os.path.join(data_dir, dataset_name, label)
        counter = 0

        for file_name in os.listdir(label_path):
            if counter >= n_images_per_label:
                break

            img_path = os.path.join(label_path, file_name)
            try:
                img = image.load_img(img_path, target_size=image_shape)
                img_array = image.img_to_array(img) / 255.0
                X.append(img_array)
                y.append(label)
                counter += 1
            except:
                continue

    X = np.array(X)
    y = np.array(y)

    if cache:
        joblib.dump((X, y), cache_file)
        print(f"Saved cache to {cache_file}")

    return X, y


Loading into Array

In [ ]:
X_walls, y_walls = load_image_as_array(train_path, dataset_name='Walls', image_shape=(256, 256), n_images_per_label=30)

print("Loaded shape:", X_walls.shape)
print("Labels:", np.unique(y_walls))

In [ ]:
def plot_mean_variability_per_labels(X, y, dataset_name, figsize=(12, 5), save_image=False):
    for label in np.unique(y):
        sns.set_style("white")

        mask = y == label
        images = X[mask]

        avg_img = np.mean(images, axis=0)
        std_img = np.std(images, axis=0)

        print(f"Dataset: {dataset_name} | Class: {label}")
        fig, axes = plt.subplots(1, 2, figsize=figsize)
        axes[0].imshow(avg_img)
        axes[0].set_title(f"Mean Image — {label}")
        axes[1].imshow(std_img)
        axes[1].set_title(f"Variability — {label}")

        for ax in axes:
            ax.axis('off')

        plt.tight_layout()

        if save_image:
            filename = f"{output_path}/mean_std_{dataset_name}_{label}.png"
            plt.savefig(filename, dpi=150, bbox_inches='tight')
            print(f"Saved to {filename}")
        else:
            plt.show()


In [ ]:
for dataset in ['Walls', 'Decks', 'Pavements']:
    print(f"\n Loading and visualizing: {dataset}")
    X, y = load_image_as_array(train_path, dataset_name=dataset, image_shape=(256, 256), n_images_per_label=30)
    plot_mean_variability_per_labels(X=X, y=y, dataset_name=dataset, save_image=True)

In [ ]:
def diff_between_avg_images(X, y, label_1, label_2, dataset_name, figsize=(15, 5), save_image=False):
    """Compute and visualize the difference between average images of two labels."""
    if label_1 not in np.unique(y) or label_2 not in np.unique(y):
        print(f"Labels must be in {np.unique(y)}")
        return

    # Get images for each label
    images_1 = X[y == label_1]
    images_2 = X[y == label_2]

    # Compute mean images
    avg_img_1 = np.mean(images_1, axis=0)
    avg_img_2 = np.mean(images_2, axis=0)

    # Compute difference
    diff_img = avg_img_1 - avg_img_2

    # Plot
    fig, axes = plt.subplots(1, 3, figsize=figsize)
    axes[0].imshow(avg_img_1)
    axes[0].set_title(f"Avg: {label_1}")
    axes[1].imshow(avg_img_2)
    axes[1].set_title(f"Avg: {label_2}")
    axes[2].imshow(diff_img)
    axes[2].set_title(f"Difference: {label_1} - {label_2}")

    for ax in axes:
        ax.axis('off')

    plt.tight_layout()
    
    if save_image:
        filename = f"{output_path}/diff_{dataset_name}_{label_1}_vs_{label_2}.png"
        plt.savefig(filename, dpi=150, bbox_inches='tight')
        print(f"Saved: {filename}")
    else:
        plt.show()


In [ ]:
for dataset in ['Walls', 'Decks', 'Pavements']:
    print(f"\nComparing Cracked vs Non-Cracked in: {dataset}")
    X, y = load_image_as_array(
        data_dir=train_path,
        dataset_name=dataset,
        image_shape=(256, 256),
        n_images_per_label=30
    )
    diff_between_avg_images(
        X, y,
        label_1='Cracked',
        label_2='Non-cracked',
        dataset_name=dataset,
        save_image=True
    )

In [ ]:
import itertools

def image_montage(dir_path, dataset_name, label, nrows=3, ncols=3, figsize=(10, 10)):
    """
    Display a montage of random images for a specific dataset and label.
    """
    label_dir = os.path.join(dir_path, dataset_name, label)
    all_images = os.listdir(label_dir)

    if len(all_images) < nrows * ncols:
        print(f"Not enough images in {dataset_name}/{label} to create a {nrows}x{ncols} montage.")
        return

    sample_images = random.sample(all_images, nrows * ncols)
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)
    axes = axes.flatten()

    for idx, img_name in enumerate(sample_images):
        img_path = os.path.join(label_dir, img_name)
        img = imread(img_path)
        axes[idx].imshow(img)
        axes[idx].axis('off')
        axes[idx].set_title(f"{label}", fontsize=10)

    plt.suptitle(f"{dataset_name} — {label} (Sample Images)", fontsize=14)
    plt.tight_layout()
    plt.show()

In [ ]:
for dataset in ['Walls', 'Decks', 'Pavements']:
    for label in dataset_labels[dataset]:
        print(f"\nMontage: {dataset}/{label}")
        image_montage(train_path, dataset_name=dataset, label=label, nrows=3, ncols=3)